# 04 Comparison of results

Compares the main analyses to the results when using the alternative FDG-PET reference region  
Also compares p-tau217 results in the main analyses to the ratio of p-tau217 to amyloid-beta 42  

In [22]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import pickle

# read in and clean data

In [23]:
data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/ptau-217 and FDG PET AD - official copy/"

In [24]:
rename_columns = ['model_p_val_baseline', 'model_B_val_baseline',
'model_n_baseline', 
       'model_p_val_Intercept', 'model_B_val_Intercept',
       'model_p_val_x_var', 'model_B_val_x_var',
        'model_p_val_age','model_B_val_age',
       'model_p_val_Education_years', 'model_B_val_Education_years', 
       'model_p_val_sex[T.M]', 'model_B_val_sex[T.M]',
        'model_p_val_apoe4_carrier[T.Yes]', 'model_B_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]', 'model_B_val_race[T.White/NH]']

covariate_pval_columns = [ 'model_p_val_age',
       'model_p_val_Education_years', 
       'model_p_val_sex[T.M]', 
        'model_p_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]']


# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42', 'GFAP_aa', 'NFL_aa', 'pTau217_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 
'VOL_Inf_Mid_Fus_Temp_Rz','VOL_MidFront_Lz', 'VOL_MidFront_Rz']
      

In [25]:
# read in data
results_new_para, data4 = pd.read_pickle(code_folder_loc + '/output/01_data_new_para_rr.pkl')
results_new_para = results_new_para.rename(columns = dict(zip(rename_columns, 
['new_para_' + ele for ele in rename_columns])))

# read in data
results_pons, data2 = pd.read_pickle(code_folder_loc + '/output/01_data_pons_rr.pkl')
results_pons = results_pons.rename(columns = dict(zip(rename_columns, 
['pons_' + ele for ele in rename_columns])))

# Pons vs New Para

In [26]:
merged_results = pd.merge(results_pons, results_new_para, how = 'inner')

## x_var significance

In [27]:
# any relationships that are significant using pons rr but not significant when using main rr
merged_results.loc[(merged_results['pons_model_p_val_x_var'] < 0.05) & (merged_results['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline


In [28]:
# any relationships that are significant using main rr but not significant when using pons rr
merged_results.loc[(merged_results['pons_model_p_val_x_var'] >= 0.05) & (merged_results['new_para_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline
0,Ab42_40_log10,MTL_gm,0.538983,0.010588,0.114689,0.378426,31,35
2,Ab42_40_log10,Precun_gm,0.269296,0.021403,0.204866,0.363460,31,35
18,pTau181_log10,MTL_gm,0.103894,0.000048,0.271625,0.542955,37,41
23,pTau181_log10,PostCing_gm,0.281572,0.015868,0.159634,0.372708,37,41
26,pTau217_log10,Precun_gm,0.240472,0.043419,-0.206857,-0.333893,34,37
30,pTau231_log10,MTL_gm,0.164072,0.033270,0.247989,0.350807,33,37
38,pTau217_Ab42_log10,Precun_gm,0.400128,0.048775,-0.162343,-0.351136,29,32
39,pTau217_Ab42_log10,Par_gm,0.123870,0.007558,-0.292311,-0.463413,29,32
40,pTau217_Ab42_log10,Temp_gm,0.146871,0.025396,-0.276260,-0.394661,29,32


In [29]:
# difference in p-values across all models
(merged_results['pons_model_p_val_x_var']  - merged_results['new_para_model_p_val_x_var']).describe().round(3)

count    707.000
mean       0.011
std        0.090
min       -0.598
25%       -0.000
50%        0.000
75%        0.000
max        0.924
dtype: float64

In [30]:
# relationships where p-value changes more than 0.1
merged_results.loc[(abs(merged_results['pons_model_p_val_x_var']  - merged_results['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'pons_model_p_val_x_var',
       'pons_model_B_val_x_var', 'pons_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

,x_var,y_var,pons_model_p_val_x_var,pons_model_B_val_x_var,pons_model_n_baseline,new_para_model_p_val_x_var,new_para_model_B_val_x_var,new_para_model_n_baseline
18,pTau181_log10,MTL_gm,0.103894,0.271625,37,0.000048,0.542955,41
39,pTau217_Ab42_log10,Par_gm,0.123870,-0.292311,29,0.007558,-0.463413,32
0,Ab42_40_log10,MTL_gm,0.538983,0.114689,31,0.010588,0.378426,35
23,pTau181_log10,PostCing_gm,0.281572,0.159634,37,0.015868,0.372708,41
2,Ab42_40_log10,Precun_gm,0.269296,0.204866,31,0.021403,0.363460,35
40,pTau217_Ab42_log10,Temp_gm,0.146871,-0.276260,29,0.025396,-0.394661,32
30,pTau231_log10,MTL_gm,0.164072,0.247989,33,0.033270,0.350807,37
26,pTau217_log10,Precun_gm,0.240472,-0.206857,34,0.043419,-0.333893,37
38,pTau217_Ab42_log10,Precun_gm,0.400128,-0.162343,29,0.048775,-0.351136,32
10,GFAP_log10,Temp_gm,0.632654,-0.079818,37,0.071993,-0.283948,41


## covariate significance

In [31]:
# those where covariate significance differs between the reference regions
a_sig = merged_results[['pons_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = merged_results[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = merged_results[a_sig != b_sig].drop(columns = merged_results.columns[merged_results.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,new_para_model_p_val_Education_years,new_para_model_p_val_age,pons_model_p_val_sex[T.M],x_var,y_var
20,0.006586,0.016969,NaN,pTau181_log10,Precun_gm
0,0.002342,NaN,NaN,Ab42_40_log10,MTL_gm
6,0.005543,NaN,NaN,GFAP_log10,MTL_gm
12,0.001880,NaN,NaN,NFL_log10,MTL_gm
18,0.000017,NaN,NaN,pTau181_log10,MTL_gm
24,0.013116,NaN,NaN,pTau217_log10,MTL_gm
36,0.006126,NaN,NaN,pTau217_Ab42_log10,MTL_gm
9,NaN,0.028485,NaN,GFAP_log10,Par_gm
15,NaN,0.036415,NaN,NFL_log10,Par_gm
21,NaN,0.041938,NaN,pTau181_log10,Par_gm


# ptau 217 vs ptau 217/Ab42 (Within results when using the main reference region)

In [32]:
temp_a, data4 = pd.read_pickle(code_folder_loc + '/output/01_data_new_para_rr.pkl')
temp_b = temp_a[temp_a['x_var'].isin(['pTau217_log10']) | temp_a['y_var'].isin(['pTau217_log10'])]
temp_a = temp_a[temp_a['x_var'].isin(['pTau217_Ab42_log10']) | temp_a['y_var'].isin(['pTau217_Ab42_log10'])]
temp_a = temp_a.rename(columns = dict(zip(rename_columns, ['nonratio_' + ele for ele in rename_columns])))

temp_a['x_var'] = temp_a['x_var'].str.replace('pTau217_Ab42', 'pTau217') # change the ratio name so that they can be merged with ptau217 alone (ratio instead becomes indicated by the column name prefix)
temp_a['y_var'] = temp_a['y_var'].str.replace('pTau217_Ab42', 'pTau217')
temp_b = temp_b.rename(columns = dict(zip(rename_columns, ['ratioAb42_' + ele for ele in rename_columns])))


In [33]:
merged_results = pd.merge(temp_a, temp_b, how = 'inner').sort_values(['x_var', 'type', 'y_var'])
merged_results

,y_var,x_var,nonratio_model_p_val_baseline,nonratio_model_B_val_baseline,nonratio_model_n_baseline,nonratio_model_p_val_Intercept,nonratio_model_p_val_Education_years,nonratio_model_p_val_x_var,nonratio_model_B_val_Intercept,nonratio_model_B_val_Education_years,...,ratioAb42_model_B_val_Education_years,ratioAb42_model_B_val_x_var,ratioAb42_model_p_val_age,ratioAb42_model_B_val_age,ratioAb42_model_p_val_sex[T.M],ratioAb42_model_B_val_sex[T.M],ratioAb42_model_p_val_apoe4_carrier[T.Yes],ratioAb42_model_B_val_apoe4_carrier[T.Yes],ratioAb42_model_B_val_race[T.White/NH],ratioAb42_model_p_val_race[T.White/NH]
8,pTau217_log10,Ab42_40_log10,Intercept 1.000000 Ab42_40_log10 0.4...,Intercept 3.469447e-17 Ab42_40_log10 ...,32,1.0,NaN,0.413445,3.469447e-17,NaN,...,NaN,0.152885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,pTau217_log10,GFAP_log10,Intercept 1.000000 GFAP_log10 0.216348 ...,Intercept 3.469447e-17 GFAP_log10 2.246...,32,1.0,NaN,0.216348,3.469447e-17,NaN,...,NaN,0.381256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,pTau217_log10,NFL_log10,Intercept 1.000000 NFL_log10 0.416678 dt...,Intercept 3.469447e-17 NFL_log10 -1.48692...,32,1.0,NaN,0.416678,3.469447e-17,NaN,...,NaN,0.138020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,pTau217_log10,pTau181_log10,Intercept 1.000000 pTau181_log10 0.0...,Intercept 3.469447e-17 pTau181_log10 ...,32,1.0,NaN,0.009418,3.469447e-17,NaN,...,NaN,0.411986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,MMSE,pTau217_log10,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 4.479307e-16 pTau217_Ab4...,29,1.0,NaN,0.463867,4.479307e-16,NaN,...,NaN,-0.282524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,VOL_TotalGray_Rz,pTau217_log10,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 9.714451e-17 pTau217_Ab4...,27,1.0,NaN,0.725136,9.714451e-17,NaN,...,NaN,-0.263960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,VOL_TotalGrayz,pTau217_log10,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 5.551115e-17 pTau217_Ab4...,27,1.0,NaN,0.906817,5.551115e-17,NaN,...,NaN,-0.282799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,VOL_Ventricles_Lz,pTau217_log10,Intercept 1.000000 pTau217_Ab42_lo...,Intercept -6.938894e-18 pTau217_Ab4...,27,1.0,NaN,0.621435,-6.938894e-18,NaN,...,NaN,0.187538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,VOL_Ventricles_Rz,pTau217_log10,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 2.636780e-16 pTau217_Ab4...,27,1.0,NaN,0.297138,2.636780e-16,NaN,...,NaN,0.390285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## x_var significance

In [34]:
# models where both the ratio and p-tau217 alone are significant
merged_results.loc[(merged_results['ratioAb42_model_p_val_x_var'] < 0.05) & (merged_results['nonratio_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 
'ratioAb42_model_p_val_x_var', 'nonratio_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'nonratio_model_B_val_x_var', 
'ratioAb42_model_n_baseline','nonratio_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,nonratio_model_p_val_x_var,ratioAb42_model_B_val_x_var,nonratio_model_B_val_x_var,ratioAb42_model_n_baseline,nonratio_model_n_baseline
11,pTau181_log10,pTau217_log10,0.011288,0.009418,0.411986,0.451905,37,32
20,pTau217_log10,bvrt,0.002598,0.005452,-0.490103,-0.481772,33,28
22,pTau217_log10,tma,0.013732,0.049854,0.412498,0.381088,31,27
5,pTau217_log10,Par_gm,0.001423,0.007558,-0.505295,-0.463413,37,32
4,pTau217_log10,Precun_gm,0.043419,0.048775,-0.333893,-0.351136,37,32
6,pTau217_log10,Temp_gm,0.001592,0.025396,-0.500866,-0.394661,37,32
16,pTau217_log10,pTau181_log10,0.011288,0.009418,0.411986,0.451905,37,32


In [35]:
# models where the ratio is significant but p-tau217 alone is not
merged_results.loc[(merged_results['ratioAb42_model_p_val_x_var'] < 0.05) & (merged_results['nonratio_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'ratioAb42_model_p_val_x_var', 'nonratio_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'nonratio_model_B_val_x_var', 
'ratioAb42_model_n_baseline','nonratio_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,nonratio_model_p_val_x_var,ratioAb42_model_B_val_x_var,nonratio_model_B_val_x_var,ratioAb42_model_n_baseline,nonratio_model_n_baseline
9,GFAP_log10,pTau217_log10,0.019901,0.216348,0.381256,0.224680,37,32
21,pTau217_log10,dstotal,0.044728,0.163501,-0.346448,-0.265756,34,29
1,pTau217_log10,SensMot_gm,0.029874,0.063871,-0.357414,-0.331439,37,32
14,pTau217_log10,GFAP_log10,0.019901,0.216348,0.381256,0.224680,37,32
105,pTau217_log10,VOL_InfParSupra_Lz,0.004959,0.735875,-0.429074,-0.062011,32,27
106,pTau217_log10,VOL_InfParSupra_Rz,0.035118,0.964642,-0.373721,-0.008955,32,27
46,pTau217_log10,VOL_InfPar_Lz,0.018954,0.969799,-0.376511,0.007081,32,27
47,pTau217_log10,VOL_InfPar_Rz,0.046486,0.756790,-0.354532,-0.062501,32,27
87,pTau217_log10,VOL_Parietal_Lz,0.045699,0.482644,-0.316908,-0.125376,32,27
99,pTau217_log10,VOL_Precun_InfPar_Lz,0.028436,0.690055,-0.347563,-0.071996,32,27


In [36]:
# models where p-tau217 alone is significant but the ratio is not
merged_results.loc[(merged_results['ratioAb42_model_p_val_x_var'] >= 0.05) & (merged_results['nonratio_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'ratioAb42_model_p_val_x_var', 'nonratio_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'nonratio_model_B_val_x_var', 
'ratioAb42_model_n_baseline','nonratio_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,nonratio_model_p_val_x_var,ratioAb42_model_B_val_x_var,nonratio_model_B_val_x_var,ratioAb42_model_n_baseline,nonratio_model_n_baseline
17,pTau217_log10,pTau231_log10,0.074139,0.008019,0.310251,0.467415,34,31
74,pTau217_log10,VOL_AntCingulate_Lz,0.219084,0.043749,0.223382,0.390970,32,27
56,pTau217_log10,VOL_Insula_Lz,0.427073,0.015863,0.145435,0.459634,32,27
57,pTau217_log10,VOL_Insula_Rz,0.276686,0.019840,0.198268,0.445594,32,27
40,pTau217_log10,VOL_MidTemp_Lz,0.570045,0.009827,-0.104281,0.516237,32,27
12,pTau231_log10,pTau217_log10,0.074139,0.008019,0.310251,0.467415,34,31


In [37]:
# p-value difference accross all models
(merged_results['ratioAb42_model_p_val_x_var']  - merged_results['nonratio_model_p_val_x_var']).describe().round(3)

count    107.000
mean      -0.097
std        0.335
min       -0.951
25%       -0.270
50%       -0.061
75%        0.081
max        0.817
dtype: float64

In [38]:
# models where the difference in p-values is over 0.1
merged_results.loc[(abs(merged_results['ratioAb42_model_p_val_x_var']  - merged_results['nonratio_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'ratioAb42_model_p_val_x_var',
       'ratioAb42_model_B_val_x_var', 'ratioAb42_model_n_baseline', 'nonratio_model_p_val_x_var', 'nonratio_model_B_val_x_var',
       'nonratio_model_n_baseline']].sort_values('nonratio_model_p_val_x_var')

,x_var,y_var,ratioAb42_model_p_val_x_var,ratioAb42_model_B_val_x_var,ratioAb42_model_n_baseline,nonratio_model_p_val_x_var,nonratio_model_B_val_x_var,nonratio_model_n_baseline
40,pTau217_log10,VOL_MidTemp_Lz,0.570045,-0.104281,32,0.009827,0.516237,27
56,pTau217_log10,VOL_Insula_Lz,0.427073,0.145435,32,0.015863,0.459634,27
57,pTau217_log10,VOL_Insula_Rz,0.276686,0.198268,32,0.019840,0.445594,27
74,pTau217_log10,VOL_AntCingulate_Lz,0.219084,0.223382,32,0.043749,0.390970,27
94,pTau217_log10,VOL_InfInsFrontal_Rz,0.919390,-0.018630,32,0.101900,0.321583,27
...,...,...,...,...,...,...,...,...
89,pTau217_log10,VOL_Frontal_Lz,0.462486,-0.134652,32,0.958036,0.010629,27
36,pTau217_log10,VOL_Fusi_Lz,0.191986,-0.236770,32,0.963078,-0.009351,27
106,pTau217_log10,VOL_InfParSupra_Rz,0.035118,-0.373721,32,0.964642,-0.008955,27
46,pTau217_log10,VOL_InfPar_Lz,0.018954,-0.376511,32,0.969799,0.007081,27


## covariate significance

In [39]:
# models where covariate significance differs between p-tau217 and the ratio
a_sig = merged_results[['ratioAb42_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = merged_results[['nonratio_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = merged_results[a_sig != b_sig].drop(columns = merged_results.columns[merged_results.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,nonratio_model_p_val_Education_years,nonratio_model_p_val_age,nonratio_model_p_val_sex[T.M],ratioAb42_model_p_val_age,ratioAb42_model_p_val_apoe4_carrier[T.Yes],ratioAb42_model_p_val_sex[T.M],x_var,y_var
40,0.007396,0.017757,NaN,NaN,NaN,NaN,pTau217_log10,VOL_MidTemp_Lz
41,0.046259,NaN,0.031,NaN,NaN,NaN,pTau217_log10,VOL_MidTemp_Rz
60,NaN,0.010897,NaN,NaN,NaN,NaN,pTau217_log10,VOL_MidFront_Lz
95,NaN,0.014246,NaN,NaN,NaN,NaN,pTau217_log10,VOL_MidSupFrontal_Lz
44,NaN,0.021432,NaN,NaN,NaN,NaN,pTau217_log10,VOL_Precun_Lz
22,NaN,NaN,NaN,0.034826,NaN,NaN,pTau217_log10,tma
88,NaN,NaN,NaN,0.017752,NaN,NaN,pTau217_log10,VOL_Parietal_Rz
51,NaN,NaN,NaN,NaN,0.022287,NaN,pTau217_log10,VOL_SupraMarg_Rz
13,NaN,NaN,NaN,NaN,NaN,0.029434,pTau217_log10,Ab42_40_log10
78,NaN,NaN,NaN,NaN,NaN,0.015007,pTau217_log10,VOL_Entorhinal_Lz
